In [ ]:
import sys

from netlistio.ingestor.parser import Parser
from netlistio.ingestor.scanner import Scanner
from netlistio.ingestor.spice import SpiceChunkParserFactory, SpiceScanStrategy
from netlistio.ingestor.flattener import Flattener
from netlistio.models.spice import prefix_registry, SpiceNetlist
from netlistio.models.generic import Primitive
from netlistio.ingestor.registry import ModelRegistry
from netlistio.ingestor.linker import link

def _scanner_factory(filepath):
    return Scanner(
        filepath,
        SpiceScanStrategy(),
    )

def _parser_factory(filepath, scanner):
    return Parser(
        filepath,
        scanner,
        SpiceChunkParserFactory(),
    ) 

In [2]:
netlist_file = "/workspaces/NetlistIO/tests/data/foundry/sram/Prelayout/Spice_models/sense_amplifier.spice"
scanner = _scanner_factory(netlist_file)
print(scanner.scan())


deque([ParseRegion(filepath='/workspaces/NetlistIO/tests/data/foundry/sram/Prelayout/Spice_models/sense_amplifier.spice', start_byte=0, end_byte=707, region_type=<RegionType.GLOBAL: 1>, context_delimiter=None, context_name=None)])


In [3]:
parse_result = _parser_factory(netlist_file, scanner).parse()
for include in parse_result.includes:
    print(f"{include=}")
print(f"{len(parse_result.cells)} cells parsed.")

include=LibraryDirective(filepath='./libs/models/sky130.lib.spice', source_file='/workspaces/NetlistIO/tests/data/foundry/sram/Prelayout/Spice_models/sense_amplifier.spice', strict=True, section='tt')
14 cells parsed.


In [4]:
Flattener(
    parse_result=parse_result,
    scanner_factory=_scanner_factory,
    parser_factory=_parser_factory
).flatten()
for include in parse_result.includes:
    print(f"{include=}")
print(f"{len(parse_result.cells)} cells parsed.")

FileNotFoundError: Included file not found: fs

In [ ]:
# Add static primitives from SpicePrimitiveTypes
model_registry = ModelRegistry(
    static_primitives={
        primitive.name.lower(): primitive
        for primitive in prefix_registry().values()
        if isinstance(primitive, Primitive)
    }
)
link_result = link(parse_result, model_registry, SpiceNetlist)
netlist = link_result.netlist

In [ ]:
for cell in netlist.cells:
    cell.write(sys.stdout)

Instance: XM1
    Net: net1
    Net: blb
    Net: net2
    Net: gnd
    l=0.21, w=0.42
    Model:
        Unresolved: sky130_fd_pr__nfet_01v8
Instance: XM3
    Net: out
    Net: bl
    Net: net2
    Net: gnd
    l=0.21, w=0.42
    Model:
        Unresolved: sky130_fd_pr__nfet_01v8
Instance: XM2
    Net: net1
    Net: vdd
    l=0.21, w=0.42
    Model:
        Unresolved: sky130_fd_pr__pfet_01v8
Instance: XM4
    Net: out
    Net: net1
    Net: vdd
    l=0.21, w=0.84
    Model:
        Unresolved: sky130_fd_pr__pfet_01v8
Instance: XM5
    Net: net2
    Net: rd_en
    Net: gnd
    l=0.21, w=1.26
    Model:
        Unresolved: sky130_fd_pr__nfet_01v8
Instance: XM6
    Net: dout
    Net: out
    Net: gnd
    l=0.21, w=0.42
    Model:
        Unresolved: sky130_fd_pr__nfet_01v8
Instance: XM7
    Net: dout
    Net: out
    Net: vdd
    l=0.21, w=0.42
    Model:
        Unresolved: sky130_fd_pr__pfet_01v8
Instance: XM1
    Net: net1
    Net: blb
    Net: net2
    Net: gnd
    l=0.21, w=0.42
  